In [1]:
import pandas as pd

### Leitura dados posse

In [46]:
sheet_id = '1SsVygtWx_Tv6Ps5zjhxeNJcDincPk6jpse-O5aFHQWY'
sheet_name = 'DadosPosseTJ'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_posse = pd.read_csv(url)

C:\Users\labci\AppData\Local\Temp\ipykernel_12676\979523307.py:4: DtypeWarning: Columns (36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posse = pd.read_csv(url)


In [47]:
# Filtrar por imobiliário
df_posse = df_posse.query('Imobiliário == "Sim"')
len(df_posse)

55015

In [66]:
# Carrega dados das UTAs
utas = pd.read_csv('wkt_utas_rmsp.txt', sep='\t', usecols=['COD_UTA', 'DOM_PP'])
utas = utas.rename(columns={'COD_UTA':'UTA'})
cod_utas = utas['UTA'].to_list()

In [49]:
# Filtrar para RMSP (1a RAJ + )
df_posse = df_posse.query('UTA in @cod_utas')
len(df_posse)

21737

In [85]:
# Filtrar para depois de 2017
df_posse['Ano'] = df_posse['Data de Disponibilização'].apply(lambda x: int(x[-4:]))
df_posse = df_posse.query('Ano >= 2017')

In [86]:
def filtar_sentencas_procedentes(sentenca):
    if not sentenca or not isinstance(sentenca, str):
        return False
    sentencas = [i.strip() for i in sentenca.split(',')]
    if ('Procedente' in sentencas or 'Homologação' in sentencas) and 'Improcedente' not in sentencas:
        return True
    else:
        return False

In [87]:
# Filtrar por sentença
df_posse = df_posse[df_posse['Sentença'].apply(filtar_sentencas_procedentes)]
len(df_posse)

9018

In [88]:
df_posse.columns

Index(['Hora registro', 'Processo', 'Classe', 'Assunto', 'Magistrado',
       'Comarca', 'Foro', 'Vara', 'Data de Disponibilização', 'Ano etiqueta',
       'Caracteres sentença', 'Palavras sentença', 'Frases sentença',
       'Tipo partes', 'Parte 1', 'Parte 2', 'Razão parte 1', 'Razão parte 2',
       'Parte 2 múltipla', 'Grupo parte 1', 'Grupo parte 2',
       'Descrição parte 1', 'RAJ', 'UTA', 'Justiça gratuita',
       'Prioridade idoso', 'Aluguel (R$)', 'Aluguel atualizado (R$)',
       'Dívida do requerido (R$)', 'Aluguel NA', 'Aluguel atualizado NA',
       'Dívida do requerido NA', 'Endereço', 'Sentença', 'Contagem sentenças',
       'Ambiental', 'Auxílio emergencial', 'Covid-19', 'Idoso', 'Imobiliário',
       'Invasão', 'Sublocação', 'Uso imóvel', 'Ano'],
      dtype='object')

#### Exportação de dados

In [89]:
# Mapa dos conflitos de posse RMSP
export = df_posse.pivot_table(index='UTA', values='Processo', aggfunc='count').reset_index()
export = export.merge(utas, on='UTA')
export['Processos por domicilio'] = round(export['Processo']*100000/export['DOM_PP'],0).astype(int)
export = export.rename(columns={'Processo':'Processos'})
export = export[['UTA', 'Processos', 'Processos por domicilio']]

export.to_csv('posse_por_uta.csv', sep=';', index=False)

In [103]:
df_posse['Ano-Mês'] = df_posse['Data de Disponibilização'].apply(lambda x: x.split('/')[-1] + '-' + x.split('/')[-2]) 

In [106]:
# Linha do tempo
df_posse
export = df_posse.pivot_table(index='Ano-Mês', values='Processo', aggfunc='count').reset_index()
export = export.rename(columns = {'Processo':'Processos'})

export.to_csv('posse_por_ano-mes.csv', sep=';', index=False)

### Leitura dados despejos

sheet_id = ''
sheet_name = ''
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_posse = pd.read_csv(url)